In [1]:
# обработка данных
import pandas as pd
import numpy as np
from datetime import datetime

# сетап plot.ly - режим работы в тетрадке и импорт объектов
from plotly.offline import init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected=True)

# задачи машинного обучения
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, ParameterGrid
from sklearn import metrics

# прочее
import warnings
warnings.filterwarnings('ignore')

Загрузим данные для работы в датафрейм. Маленький фокус для правильной работы - функция для парсинга даты.

In [2]:
def date_parser(string):
    date = datetime.strptime(string, '%d.%m.%Y')
    return date

df = pd.read_csv('Тестовое задание.csv',
                 sep=';',
                 parse_dates=[0],
                 index_col=0,
                 date_parser=date_parser,
                 engine='python',
                 encoding='utf-8')

df.head()

,Продажи упаковки,Россия 1,Домашний,НТВ,Первый Канал,Пятый Канал,ТВ Центр,ТВ-3,Рен ТВ,Звезда,Нишевые каналы,Реклама в прессе,Конкурент1,Конкурент2,Конкурент3,Конкурент4,Цена бренда,Средняя цена в категории
Неделя,,,,,,,,,,,,,,,,,,
2010-01-04,7092.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,104
2010-01-11,8664.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52,100
2010-01-18,7526.0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,0,53,97
2010-01-25,9165.0,0,0,0,0,0,0,0,0,0,0,56,0,0,0,0,55,103
2010-02-01,8713.0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,52,101


In [3]:
def plotly_df(df, title = ''):
    '''
    Наивная рисовалка графиков из датафрейма
    '''
    data = []
    
    for column in df.columns:
        trace = go.Scatter(x=df.index,
                          y=df[column],
                          mode='lines',
                          name=column)
        data.append(trace)
        
    layout = dict(title=title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

In [4]:
sales = df[['Продажи упаковки']]
sales = sales[sales['Продажи упаковки'] > 0]
plotly_df(sales, title='Продажи')

Из графика хорошо видно, что временной ряд не является стационарным, т.к.:
    - большой разброс значений
    - значения становятся ближе друг к другу
    
Поэтому мы не будем использовать ARMA-based модели, и имеет смысл рассматривать скользящее среднее, взвешенное среднее, экпоненциальное сглаживание и двойное экспоненциальное сглаживание, при рассмотре данных как временного ряда для анализа ближайших трендов и использовать регрессию для построения прогнозов.

In [5]:
df.describe()

,Продажи упаковки,Россия 1,Домашний,НТВ,Первый Канал,Пятый Канал,ТВ Центр,ТВ-3,Рен ТВ,Звезда,Нишевые каналы,Реклама в прессе,Конкурент1,Конкурент2,Конкурент3,Конкурент4,Цена бренда,Средняя цена в категории
count,156.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,10028.660256,12.569231,1.830769,1.451282,6.328205,1.158974,0.671795,0.400000,0.225641,0.292308,0.912821,11.276923,4.866667,2.989744,3.502564,2.635897,79.333333,130.230769
std,2071.714208,21.652101,4.125240,7.189012,17.396379,4.694849,2.748003,1.972491,1.624668,1.468463,3.237600,19.666687,13.373411,6.145143,9.043637,5.212568,14.878431,16.511261
min,6077.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.000000,97.000000
25%,8399.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68.000000,117.000000
50%,9744.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,133.000000
75%,11290.000000,36.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,3.000000,93.000000,143.000000
max,15715.000000,100.000000,22.000000,50.000000,74.000000,29.000000,18.000000,12.000000,15.000000,9.000000,20.000000,100.000000,77.000000,22.000000,62.000000,27.000000,104.000000,158.000000


Добавим к исходным данным дополнительные - номер недели в году и лаги с 4 (месяц) по 12 (квартал) 

In [6]:
data = df.copy()

Построим прогнозы, основанные на скользящем среднем.

In [7]:
sales['Скользящее среднее'] = sales['Продажи упаковки'].rolling(window=12).mean()
plotly_df(sales, title='Скользящее среднее')

In [8]:
def exponental_smoothing(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n - 1])
    return result

In [9]:
alphas = [0.3, 0.5, 0.7]

for alpha in alphas:
    sales[f'Alpha = {alpha}'] = exponental_smoothing(sales['Продажи упаковки'], alpha)

plotly_df(sales.drop('Скользящее среднее', axis=1), title='Экспоненциальное сглаживание')

Примечание - если на легенде нажать на линию, она изчезнет, можно сравнивать меньшее количество графиков.

В качестве модели будем использовать линейную регрессию, добавив некоторые дополнительные признаки - неделю, значение продаж неделю, месяц, квартал назад. Для выбора лучшей модели воспользуемся классом GridSearchCV, который принимает на вход изменяемые параметры для модели и отдает на выход оптимальные параметры.

В качестве метрики будем использовать r2_score, т.к. эта метрика позволяет оценить качество модели для прогнозов.

Больших лагов добавлять не будем, т.к. теряется очень большая часть выборки.

In [10]:
data['Неделя'] = data.index.week
learning_samples = data.copy()

for i in range(4, 13):
    learning_samples[f'Lag {i}'] = learning_samples['Продажи упаковки'].shift(i)

learning_samples = learning_samples.dropna()
learning_samples.head()

,Продажи упаковки,Россия 1,Домашний,НТВ,Первый Канал,Пятый Канал,ТВ Центр,ТВ-3,Рен ТВ,Звезда,...,Неделя,Lag 4,Lag 5,Lag 6,Lag 7,Lag 8,Lag 9,Lag 10,Lag 11,Lag 12
Неделя,,,,,,,,,,,,,,,,,,,,,
2010-03-29,7389.0,0,0,0,0,0,0,0,0,0,...,13,9154.0,7798.0,8595.0,7489.0,8713.0,9165.0,7526.0,8664.0,7092.0
2010-04-05,8215.0,0,0,0,0,0,0,0,0,0,...,14,8317.0,9154.0,7798.0,8595.0,7489.0,8713.0,9165.0,7526.0,8664.0
2010-04-12,8121.0,0,0,0,0,0,0,0,0,0,...,15,8671.0,8317.0,9154.0,7798.0,8595.0,7489.0,8713.0,9165.0,7526.0
2010-04-19,8120.0,0,0,0,0,0,0,0,0,0,...,16,8837.0,8671.0,8317.0,9154.0,7798.0,8595.0,7489.0,8713.0,9165.0
2010-04-26,7494.0,0,0,0,0,0,0,0,0,0,...,17,7389.0,8837.0,8671.0,8317.0,9154.0,7798.0,8595.0,7489.0,8713.0


In [11]:
learning_samples.tail()

,Продажи упаковки,Россия 1,Домашний,НТВ,Первый Канал,Пятый Канал,ТВ Центр,ТВ-3,Рен ТВ,Звезда,...,Неделя,Lag 4,Lag 5,Lag 6,Lag 7,Lag 8,Lag 9,Lag 10,Lag 11,Lag 12
Неделя,,,,,,,,,,,,,,,,,,,,,
2012-11-26,8032.0,0,0,0,0,13,9,0,0,0,...,48,8649.0,8401.0,9696.0,9226.0,8606.0,8001.0,8056.0,8568.0,8887.0
2012-12-03,8118.0,0,0,0,0,16,9,0,0,0,...,49,8140.0,8649.0,8401.0,9696.0,9226.0,8606.0,8001.0,8056.0,8568.0
2012-12-10,8985.0,40,0,0,0,0,0,0,0,0,...,50,8654.0,8140.0,8649.0,8401.0,9696.0,9226.0,8606.0,8001.0,8056.0
2012-12-17,7780.0,58,0,0,0,0,0,0,0,0,...,51,8530.0,8654.0,8140.0,8649.0,8401.0,9696.0,9226.0,8606.0,8001.0
2012-12-24,7908.0,0,0,0,0,0,0,0,0,0,...,52,8032.0,8530.0,8654.0,8140.0,8649.0,8401.0,9696.0,9226.0,8606.0


In [12]:
len(learning_samples)

144

In [13]:
X_train = learning_samples[:116].drop('Продажи упаковки', axis=1)
y_train = learning_samples[:116]['Продажи упаковки']
X_test = learning_samples[116:].drop('Продажи упаковки', axis=1)
y_test = learning_samples[116:]['Продажи упаковки']

In [14]:
lin = LinearRegression()
lin.fit(X_train, y_train)

lin_predict = lin.predict(X_test)

In [15]:
plotly_df(pd.DataFrame({'Исходные данные': y_test, 'Прогноз': lin_predict}, index=learning_samples.index[116:]),
          title='Линейная регрессия без регуляризации')

На графике видно, что модель неплохо обучилась, но случаются выбросы. Попробуем добавить регуляризацию.

In [16]:
param_grid = {'alpha': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100],
              'solver': ['sag', 'lsqr']}

ridge = Ridge()
ridge_cv = GridSearchCV(ridge,
                     param_grid=param_grid)
ridge_cv.fit(X_train, y_train)
ridge_predict = ridge_cv.predict(X_test)

plotly_df(pd.DataFrame({'Исходные данные': y_test, 'Прогноз': ridge_predict}, index=learning_samples.index[116:]),
          title=f'Ridge')

In [17]:
for param, value in ridge_cv.best_params_.items():
    print(f'{param}: {value}')
    
print(f'R2 score: {metrics.r2_score(y_test, ridge_predict)}')

alpha: 100
solver: lsqr
R2 score: -0.5843955138314172


Видно, что модель сильно регуляризуется кросс-валидацией, посмотрим, как она ведет себя на остальном временном ряде. Кстати, напишем функцию для прогноза - на вход будем подавать ей обученную модель и исходные данные для прогноза, и будем в процессе выполнения функции извлекать лаги.

In [18]:
param_grid = {'alpha': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]}

lasso = Lasso()
lasso_cv = GridSearchCV(lasso,
                     param_grid=param_grid)
lasso_cv.fit(X_train, y_train)
lasso_predict = lasso_cv.predict(X_test)

plotly_df(pd.DataFrame({'Исходные данные': y_test, 'Прогноз': lasso_predict}, index=learning_samples.index[116:]),
          title=f'Lasso')

sklearn настаивает на большей регуляризации и увеличении alpha. Посмотрим подобранные значения.

In [19]:
for param, value in lasso_cv.best_params_.items():
    print(f'{param}: {value}')
    
print(f'R2 score: {metrics.r2_score(y_test, lasso_predict)}')

alpha: 100
R2 score: -0.5954442758819163


Повторим процедуру для elastic net

In [20]:
param_grid = {'alpha': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100],
              'l1_ratio': [0, 0.01, 0.03, 0.1, 0.3]}

elastic_net = ElasticNet()
elastic_net_cv = GridSearchCV(elastic_net,
                     param_grid=param_grid)
elastic_net_cv.fit(X_train, y_train)
elastic_net_predict = elastic_net_cv.predict(X_test)

plotly_df(pd.DataFrame({'Исходные данные': y_test, 'Прогноз': elastic_net_predict}, index=learning_samples.index[116:]),
          title='Elastic Net')

In [21]:
for param, value in elastic_net_cv.best_params_.items():
    print(f'{param}: {value}')
print(f'R2 score: {metrics.r2_score(y_test, elastic_net_predict)}')

alpha: 100
l1_ratio: 0
R2 score: -0.2614545934838919


Из результатов видно, что лучший результат на тестовой выборке показала elastic_net. Будем использовать эту модель для прогноза.

In [22]:
def forecast(num_casts, model, dataframe):
    df = dataframe.copy()
    index = df.index
    pred_start = len(df) - num_casts
    for x in range(num_casts):
        # обновим лаги
        for i in range(4, 13):
            df.set_value(index[pred_start + x], f'Lag {i}', df.iloc[x - num_casts - i]['Продажи упаковки'])
        # получим предикт
        X = df.iloc[x - num_casts][1:]
        df.set_value(index[pred_start + x], 'Продажи упаковки', model.predict(X))
    return df

In [23]:
for i in range(4, 13):
    data[f'Lag {i}'] = data['Продажи упаковки'].shift(i)
    
results = forecast(39, elastic_net_cv, data)

In [24]:
def plot_results():
    data = []
    fcast = go.Scatter(y=results['Продажи упаковки'][-39:],
                       x=results['Продажи упаковки'][-39:].index,
                       name='Прогноз')
    data.append(fcast)
    
    start_data = go.Scatter(x=sales.index,
                            y=sales['Продажи упаковки'],
                            name='Исходные данные')
    data.append(start_data)
    fig = dict(data=data, layout={'title': 'Прогноз'})
    iplot(fig, show_link=False)

In [25]:
plot_results()

In [26]:
results.to_csv('Результаты.csv', encoding='utf-8')